<a href="https://colab.research.google.com/github/m-o-d-e/nlp/blob/main/TP3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Charger le Dataset

In [ ]:

df = pd.read_csv('movie_review.csv')


texts = df['text'].str.lower()
tags = df['tag']

# Pré-traitement

In [ ]:
import pandas as pd
import string
import regex as re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')

# Function to preprocess a single text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]

    return filtered_words

# Read your CSV file
df = pd.read_csv('movie_review.csv')

# Apply the preprocessing function to each text in the 'text' column
tokenized_texts = df['text'].apply(preprocess_text)

# Display the preprocessed texts
print(tokenized_texts.head())





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    [films, adapted, comic, books, plenty, success...
1    [starters, created, alan, moore, eddie, campbe...
2    [say, moore, campbell, thoroughly, researched,...
3    [book, graphic, novel, 500, pages, long, inclu...
4                 [words, dont, dismiss, film, source]
Name: text, dtype: object


# Entraînement du Modèle Word2Vec


In [ ]:
# Train Word2Vec model
word2vec_model = Word2Vec(tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)

# Vectorisation des Textes

In [ ]:
import numpy as np

# Function to calculate the average Word2Vec vector for a list of words
def average_word2vec(words, model, num_features):
    feature_vector = np.zeros((num_features,), dtype="float32")
    num_words = 0

    for word in words:
        if word in model.wv.index_to_key:
            num_words += 1
            feature_vector = np.add(feature_vector, model.wv[word])

    if num_words > 0:
        feature_vector = np.divide(feature_vector, num_words)

    return feature_vector

# Apply the average Word2Vec function to each tokenized review
vectorized_reviews = tokenized_texts.apply(lambda x: average_word2vec(x, word2vec_model, 100))

# Display the vectorized reviews
print(vectorized_reviews.head())


0    [-0.32529518, 0.4411588, -0.05114542, 0.106817...
1    [-0.2055158, 0.33180833, 0.010003735, 0.263095...
2    [-0.24415465, 0.38815334, 0.048941948, 0.14001...
3    [-0.24084467, 0.36071184, -0.017482636, 0.1574...
4    [-0.35556117, 0.44850984, -0.29013237, 0.12369...
Name: text, dtype: object


# Division des Données

In [ ]:
tags = df['tag']
# Convert vectorized_reviews to a NumPy array
X = np.array(list(vectorized_reviews))

# Flatten the vectors into a 2D array
X_flat = np.array([vector.flatten() for vector in X])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_flat, tags, test_size=0.2, random_state=42)

# Construction et Évaluation du modéle

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report



# Initialize the SVM classifier
svm_clf = SVC()

# Train the SVM model
svm_clf.fit(X_train, y_train)

# Make predictions on the test set
svm_predictions = svm_clf.predict(X_test)

# Evaluate the SVM model
print("SVM Classification Report:")
print(classification_report(y_test, svm_predictions))


SVM Classification Report:
              precision    recall  f1-score   support

         neg       0.57      0.09      0.16      6366
         pos       0.57      0.95      0.71      8005

    accuracy                           0.57     14371
   macro avg       0.57      0.52      0.43     14371
weighted avg       0.57      0.57      0.46     14371

